# 12.3 OpenAI Functions Agent

이번에는 OpenAI Functions Agent를 배워보자.

먼저, Pydantic에 대해 배워보고 넘어가자.  



## Pydantic이란?
파이썬에서 데이터의 형태를 검증하고 자동으로 변환해주는 라이브러리입니다.

## 기본 사용법
````python
from pydantic import BaseModel

# 데이터 모델 정의
class 사용자(BaseModel):
    이름: str
    나이: int
    이메일: str
    활성상태: bool = True  # 기본값 설정

# 사용 예시
새사용자 = 사용자(
    이름="김철수",
    나이="30",  # 문자열이지만 자동으로 숫자로 변환됨
    이메일="kim@example.com"
)

print(새사용자.나이)  # 30 (int 타입)
print(type(새사용자.나이))  # <class 'int'>
````

## 주요 특징
1. **자동 타입 변환**: 문자열 "30"을 자동으로 정수 30으로 변환
2. **데이터 검증**: 잘못된 데이터 형식이면 오류 발생
3. **기본값 설정**: 값이 없으면 기본값 사용

## 실제 활용
- **API 요청/응답 검증**: 클라이언트에서 받은 데이터 검증
- **설정 파일 관리**: 환경 변수나 설정 파일 데이터 검증
- **데이터베이스 모델**: DB에서 가져온 데이터 검증

## 장점
- 코드가 간결해짐
- 타입 오류를 미리 방지
- 데이터 변환 작업 자동화

Pydantic은 "데이터가 예상대로 생겼는지 확인하고, 필요하면 알맞게 변환해주는 도우미"라고 생각하면 됩니다.


In [3]:
from langchain.agents import initialize_agent, AgentType
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from typing import Any, Type 
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)


class CalculatorToolArgsSchema(BaseModel):
    a: float = Field(description="The First number")
    b: float = Field(description="The second number")

class CalculatorTool(BaseTool):
    name = "CalculatorTool"
    description = """
    Use this to perform sums of two numbers.
    The first and second arguments should be numbers.
    Only receives two arguments.
    """
    args_schema : Type[CalculatorToolArgsSchema] = CalculatorToolArgsSchema

    def _run(self, a, b):
        return a + b

agent = initialize_agent(
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    verbose=True,
    tools=[
        CalculatorTool(),
    ],
)

prompt = "Tell me Total Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

agent.invoke(prompt)




> Entering new AgentExecutor chain...

Invoking: `CalculatorTool` with `{'a': 355.39, 'b': 924.87}`


1280.26
Invoking: `CalculatorTool` with `{'a': 1280.26, 'b': 721.2}`


2001.46
Invoking: `CalculatorTool` with `{'a': 2001.46, 'b': 1940.29}`


3941.75
Invoking: `CalculatorTool` with `{'a': 3941.75, 'b': 573.63}`


4515.38
Invoking: `CalculatorTool` with `{'a': 4515.38, 'b': 65.72}`


4581.1
Invoking: `CalculatorTool` with `{'a': 4581.1, 'b': 35.0}`


4616.1
Invoking: `CalculatorTool` with `{'a': 4616.1, 'b': 552.0}`


5168.1
Invoking: `CalculatorTool` with `{'a': 5168.1, 'b': 76.16}`


5244.26
Invoking: `CalculatorTool` with `{'a': 5244.26, 'b': 29.12}`


5273.38The total cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12 is $5273.38.

> Finished chain.


{'input': 'Tell me Total Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12',
 'output': 'The total cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12 is $5273.38.'}

# OpenAI Functions Agent 코드 분석

## 1. 전체 코드 개요

```python
from langchain.agents import initialize_agent, AgentType
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from typing import Any, Type 
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)


class CalculatorToolArgsSchema(BaseModel):
    a: float = Field(description="The First number")
    b: float = Field(description="The second number")

class CalculatorTool(BaseTool):
    name = "CalculatorTool"
    description = """
    Use this to perform sums of two numbers.
    The first and second arguments should be numbers.
    Only receives two arguments.
    """
    args_schema : Type[CalculatorToolArgsSchema] = CalculatorToolArgsSchema

    def _run(self, a, b):
        return a + b

agent = initialize_agent(
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    verbose=True,
    tools=[
        CalculatorTool(),
    ],
)

prompt = "Tell me Total Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

agent.invoke(prompt)
```

이 코드는 OpenAI Functions 기반 에이전트를 구현한 것입니다. 이 에이전트는 두 숫자를 더하는 계산기 도구를 사용해 여러 금액의 총합을 계산하는 데 사용됩니다.

## 2. 모듈 임포트 설명

```python
from langchain.agents import initialize_agent, AgentType
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from typing import Any, Type 
from langchain.chat_models import ChatOpenAI
```

각 임포트 항목의 역할:

- **initialize_agent**: 에이전트를 생성하는 함수입니다. LLM과 도구를 결합해 에이전트 객체를 만듭니다.
- **AgentType**: 어떤 유형의 에이전트를 사용할지 지정하는 열거형(Enum)입니다. `OPENAI_FUNCTIONS`는 OpenAI의 함수 호출 기능을 활용하는 에이전트 유형입니다.
- **BaseTool**: 도구 클래스를 생성할 때 상속받는 기본 클래스입니다. 에이전트가 사용할 도구의 인터페이스를 정의합니다.
- **BaseModel, Field**: Pydantic 라이브러리에서 가져온 클래스로, 데이터 검증과 메타데이터 정의에 사용됩니다.
- **Any, Type**: Python 타입 힌트를 위한 클래스입니다. `Type`은 클래스 자체를 타입으로 표현할 때 사용합니다.
- **ChatOpenAI**: LangChain에서 OpenAI의 채팅 모델(GPT-3.5/4)에 접근하기 위한 클래스입니다.

## 3. LLM 초기화

```python
llm = ChatOpenAI(temperature=0.1)
```

- **ChatOpenAI**: OpenAI의 채팅 모델(GPT 모델)을 사용하기 위한 래퍼 클래스입니다.
- **temperature=0.1**: 생성되는 텍스트의 무작위성을 제어하는 매개변수입니다. 낮은 값(0.1)은 더 결정적이고 일관된 출력을 생성합니다. 계산과 같은 정확성이 필요한 작업에 적합합니다.

## 4. 도구 인자 스키마 정의

```python
class CalculatorToolArgsSchema(BaseModel):
    a: float = Field(description="The First number")
    b: float = Field(description="The second number")
```

이 클래스는 계산기 도구에 전달될 인자의 구조와 타입을 정의합니다:

- **BaseModel 상속**: Pydantic의 BaseModel을 상속받아 데이터 검증 기능을 활용합니다.
- **a, b**: 두 개의 필드(인자)를 정의합니다. 두 값 모두 `float` 타입이어야 합니다.
- **Field()**: 필드에 대한 메타데이터를 제공합니다. 여기서는 설명을 추가합니다.
- **description**: 각 필드에 대한 설명으로, LLM이 이 도구를 어떻게 사용해야 하는지 이해하는 데 도움을 줍니다.

이 스키마는 에이전트가 도구를 사용할 때:
1. 두 개의 숫자 인자가 필요하다는 것을 알려줍니다
2. 자동으로 타입 검증을 제공합니다 (문자열이 전달되어도 자동으로 float로 변환)
3. 잘못된 타입이 전달되면 오류를 발생시킵니다

## 5. 계산기 도구 클래스 정의

```python
class CalculatorTool(BaseTool):
    name = "CalculatorTool"
    description = """
    Use this to perform sums of two numbers.
    The first and second arguments should be numbers.
    Only receives two arguments.
    """
    args_schema : Type[CalculatorToolArgsSchema] = CalculatorToolArgsSchema

    def _run(self, a, b):
        return a + b
```

이 클래스는 실제 에이전트가 사용할 도구를 정의합니다:

- **BaseTool 상속**: LangChain의 기본 도구 클래스를 상속받아 도구 인터페이스를 구현합니다.
- **name**: 도구의 이름을 정의합니다. 에이전트는 이 이름으로 도구를 참조합니다.
- **description**: 이 도구가 무엇을 하는지, 어떻게 사용해야 하는지 설명합니다. LLM이 도구 사용 방법을 이해하는 데 중요합니다.
- **args_schema**: 인자의 구조를 정의합니다. `Type[CalculatorToolArgsSchema]`는 클래스 자체를 타입으로 지정한다는 의미입니다.
- **_run 메소드**: 도구가 실행될 때 호출되는 메소드입니다. 두 숫자를 받아 합을 반환합니다.

## 6. 에이전트 초기화

```python
agent = initialize_agent(
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    verbose=True,
    tools=[
        CalculatorTool(),
    ],
)
```

에이전트를 초기화하는 이 코드에서:

- **llm=llm**: 에이전트가 사용할 언어 모델을 지정합니다.
- **agent=AgentType.OPENAI_FUNCTIONS**: OpenAI의 Function Calling 기능을 활용하는 에이전트 유형을 사용합니다. 이는 LLM이 명시적으로 함수를 호출할 수 있게 해주는 특별한 형태의 에이전트입니다.
- **handle_parsing_errors=True**: 에이전트가 도구의 출력을 해석하는 과정에서 오류가 발생하면 자동으로 처리하도록 합니다.
- **verbose=True**: 에이전트의 사고 과정과 도구 사용을 자세히 출력합니다. 디버깅과 이해에 유용합니다.
- **tools=[CalculatorTool()]**: 에이전트가 사용할 수 있는 도구 목록을 제공합니다. 여기서는 계산기 도구 하나만 제공됩니다.

## 7. 프롬프트 정의와 에이전트 실행

```python
prompt = "Tell me Total Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

agent.invoke(prompt)
```

- **prompt**: 에이전트에게 주어질 문제를 정의합니다. 여러 금액의 합을 계산하는 요청입니다.
- **agent.invoke(prompt)**: 정의된 프롬프트로 에이전트를 실행합니다. 에이전트는 이 질문을 분석하고, 적절한 도구(계산기)를 선택하여 문제를 해결합니다.

## 8. OpenAI Functions Agent 내부 작동 방식

여기서 사용된 `OPENAI_FUNCTIONS` 에이전트 유형은 일반 에이전트와 다른 방식으로 작동합니다:

1. **함수 선언**: 에이전트는 도구를 OpenAI Function Calling 형식으로 변환합니다. 도구의 `args_schema`가 JSON 스키마로 변환됩니다.

2. **함수 호출 결정**: 프롬프트가 LLM에 전달되면, LLM은 어떤 함수를 호출해야 할지와 어떤 인자를 사용할지 결정합니다.

3. **단계적 계산**: 계산기 도구는 한 번에 두 숫자만 더할 수 있으므로, 에이전트는 다음과 같은 단계로 문제를 해결합니다:
   - $355.39 + $924.87 = $1280.26
   - $1280.26 + $721.2 = $2001.46
   - $2001.46 + $1940.29 = $3941.75
   - ...
   - 모든 숫자를 더할 때까지 계속

4. **결과 조합**: 최종적으로 모든 계산이 완료되면, 에이전트는 최종 결과를 반환합니다.

이 방식은 일반 에이전트와 달리 ReAct(Reasoning + Acting) 패턴을 명시적으로 보여주지 않지만, 내부적으로는 함수 호출과 결과를 통해 추론과 행동을 수행합니다.

## 9. 이 구현의 장단점

### 장점:
- **타입 안전성**: Pydantic을 통해 인자 타입을 검증하고 변환합니다.
- **명확한 도구 정의**: 도구의 역할과 사용법이 명확하게 정의됩니다.
- **정확한 계산**: LLM이 직접 계산하지 않고 실제 Python 함수를 사용하여 계산하므로 정확합니다.

### 단점:
- **반복적 계산**: 계산기가 두 숫자만 처리할 수 있어 여러 단계의 계산이 필요합니다.
- **효율성**: 더 많은 숫자를 더할수록 함수 호출 횟수가 늘어납니다.

## 10. 가능한 개선 방안

더 효율적인 구현을 위해 다음과 같은 개선이 가능합니다:

```python
class AdvancedCalculatorToolArgsSchema(BaseModel):
    numbers: list[float] = Field(description="List of numbers to sum")

class AdvancedCalculatorTool(BaseTool):
    name = "AdvancedCalculatorTool"
    description = "Use this to sum a list of numbers at once."
    args_schema : Type[AdvancedCalculatorToolArgsSchema] = AdvancedCalculatorToolArgsSchema

    def _run(self, numbers):
        return sum(numbers)
```

이렇게 하면 에이전트가 모든 숫자를 한 번에 더할 수 있으므로 효율성이 크게 향상됩니다.

## 결론

이 코드는 OpenAI Function Calling을 활용한 에이전트의 구현 예시입니다. Pydantic을 통한 데이터 검증, 도구 정의, 그리고 에이전트 초기화까지 LangChain의 주요 기능들을 보여줍니다. 이러한 접근 방식은 LLM이 직접 수행하기 어려운 정확한 계산 작업을 위임하여 더 정확한 결과를 얻을 수 있게 합니다.
